# 🎯 ConvNeXtPose Testing en Kaggle - Modelos L y M

Este notebook evalúa los modelos ConvNeXtPose L y M en Human3.6M Protocol 2.

**Datasets requeridos:**
- Human3.6M Dataset (con S9_ACT2_i6, S11_ACT2_i6, annotations)
- ConvNeXtPose Pre-trained Models (checkpoints .tar)

**GPU recomendada:** T4 x2 o P100

## 📦 PASO 1: Setup Inicial

In [ ]:
# Clonar repositorio
!git clone https://github.com/EstebanCabreraArbizu/ConvNeXtPose.git
%cd ConvNeXtPose

# Verificar versiones
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 🗂️ PASO 2: Configurar Dataset Human3.6M

**IMPORTANTE:** Solo enlazamos el contenido del dataset dentro de `data/Human36M/`.
Los módulos Python originales (`dataset.py`, `Human36M.py`) permanecen intactos.

In [ ]:
# IMPORTANTE: Ajusta este path al nombre real de tu dataset en Kaggle
KAGGLE_DATASET_PATH = '/kaggle/input/human36m-dataset'  # ← CAMBIAR SEGÚN TU DATASET

# Verificar que el dataset existe
import os
if not os.path.exists(KAGGLE_DATASET_PATH):
    print(f"❌ Dataset no encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Datasets disponibles:")
    !ls /kaggle/input/
    raise FileNotFoundError("Verifica el nombre del dataset y actualiza KAGGLE_DATASET_PATH")
else:
    print(f"✓ Dataset encontrado en {KAGGLE_DATASET_PATH}")
    print("\n📂 Contenido:")
    !ls {KAGGLE_DATASET_PATH}

### (Opcional) Diagnosticar Estructura del Dataset

Si tienes dudas sobre la estructura de tu dataset, ejecuta esta celda primero para ver cómo están organizadas las carpetas.

In [ ]:
# Diagnosticar estructura del dataset (útil para identificar carpetas anidadas)
!python diagnose_kaggle_dataset.py {KAGGLE_DATASET_PATH}

In [ ]:
# Ejecutar script de configuración
# IMPORTANTE: Esto enlaza el dataset DENTRO de data/Human36M/, NO reemplaza data/
# El script detecta automáticamente carpetas anidadas (ej: annotations (1)/annotations/)
!python setup_kaggle_dataset.py --kaggle-input {KAGGLE_DATASET_PATH} --project-root /kaggle/working/ConvNeXtPose

print("\n✅ Dataset enlazado en data/Human36M/")
print("✅ Módulos Python originales intactos en data/")
print("✅ Carpetas anidadas detectadas automáticamente")

In [ ]:
# Verificar que la estructura es correcta
!python setup_kaggle_dataset.py --verify /kaggle/working/ConvNeXtPose

## 🎯 PASO 3: Preparar Checkpoints

Extraer los modelos pre-entrenados.

In [ ]:
# IMPORTANTE: Si usas gdown, ajusta la ruta al directorio donde descargaste
# Si usas dataset de Kaggle, usa: MODELS_DATASET_PATH = '/kaggle/input/convnextpose-models'
MODELS_DATASET_PATH = '/kaggle/working/ConvNeXtPose/models_tar'  # ← CAMBIAR SEGÚN TU CASO

# Verificar modelos disponibles
if os.path.exists(MODELS_DATASET_PATH):
    print("📦 Modelos disponibles:")
    !ls -lh {MODELS_DATASET_PATH}/*.tar
else:
    print(f"❌ Dataset de modelos no encontrado: {MODELS_DATASET_PATH}")
    print("\n📂 Rutas disponibles:")
    !ls -lh /kaggle/working/ConvNeXtPose/

In [ ]:
import os
import zipfile
import shutil
import tarfile
import torch

# Crear directorio para modelos
os.makedirs('output/model_dump', exist_ok=True)

# Función para extraer y convertir checkpoints
def extract_checkpoint(tar_path, model_name, expected_epoch=None):
    """Extrae checkpoint desde .tar/.zip y lo convierte en archivo .pth válido
    
    Los archivos .tar de ConvNeXtPose son ZIP con estructura de directorio en formato legacy.
    Usamos una conversión simple: empaquetar el directorio como TAR que PyTorch puede leer.
    """
    if not os.path.exists(tar_path):
        print(f"⚠️  Modelo {model_name} no encontrado en {tar_path}")
        return None
    
    print(f"📦 Extrayendo modelo {model_name} desde {tar_path}...")
    
    # Verificar tamaño del archivo
    file_size = os.path.getsize(tar_path)
    print(f"   📏 Tamaño del archivo: {file_size / (1024*1024):.2f} MB")
    
    # Leer primeros bytes para diagnosticar el formato
    with open(tar_path, 'rb') as f:
        header = f.read(512)
        print(f"   🔍 Primeros bytes (hex): {header[:50].hex()}")
    
    # Si el archivo es muy pequeño o empieza con HTML, es un error de descarga
    if file_size < 1000 or header.startswith(b'<!DOCTYPE') or header.startswith(b'<html'):
        print(f"   ❌ ERROR: El archivo parece ser HTML (error de descarga)")
        print(f"   💡 Solución: Revisa que el folder de Google Drive sea público")
        return None
    
    # Extraer a directorio temporal
    temp_dir = 'output/model_dump/temp_extract'
    os.makedirs(temp_dir, exist_ok=True)
    
    # Detectar formato: intentar ZIP primero, luego TAR
    extracted = False
    try:
        with zipfile.ZipFile(tar_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
            files = zip_ref.namelist()
            print(f"   ✓ Formato: ZIP - Extraído: {len(files)} archivos")
            extracted = True
    except zipfile.BadZipFile:
        try:
            with tarfile.open(tar_path, 'r') as tar_ref:
                tar_ref.extractall(temp_dir)
                files = tar_ref.getnames()
                print(f"   ✓ Formato: TAR - Extraído: {len(files)} archivos")
                extracted = True
        except (tarfile.ReadError, Exception) as e:
            print(f"   ❌ ERROR: No se pudo extraer el archivo")
            print(f"   💡 Formato no reconocido: {type(e).__name__}: {e}")
            shutil.rmtree(temp_dir, ignore_errors=True)
            return None
    
    if not extracted:
        shutil.rmtree(temp_dir, ignore_errors=True)
        return None
    
    # Buscar la carpeta del checkpoint (snapshot_XX.pth/ o archive/)
    found_checkpoints = []
    for item in os.listdir(temp_dir):
        item_path = os.path.join(temp_dir, item)
        if os.path.isdir(item_path):
            # Verificar que tenga data.pkl (indicador de checkpoint PyTorch)
            if os.path.exists(os.path.join(item_path, 'data.pkl')):
                found_checkpoints.append((item, item_path))
                print(f"   ✓ Checkpoint encontrado: {item}/")
    
    if not found_checkpoints:
        print(f"   ❌ No se encontró estructura de checkpoint válida")
        print(f"   📂 Contenido extraído:")
        for item in os.listdir(temp_dir)[:10]:
            print(f"      - {item}")
        shutil.rmtree(temp_dir, ignore_errors=True)
        return None
    
    # Convertir el checkpoint de directorio a archivo .pth.tar
    epoch = None
    for ckpt_name, ckpt_path in found_checkpoints:
        # Determinar el epoch desde el nombre
        import re
        match = re.search(r'snapshot_(\d+)', ckpt_name)
        if match:
            epoch = match.group(1)
            final_name = f'snapshot_{epoch}.pth.tar'
        else:
            # Si no tiene snapshot_XX, usar archive/ con epoch esperado
            if expected_epoch:
                epoch = str(expected_epoch)
                final_name = f'snapshot_{epoch}.pth.tar'
            else:
                epoch = '0'
                final_name = f'{model_name}_checkpoint.pth.tar'
        
        dest_path = os.path.join('output/model_dump', final_name)
        
        print(f"   🔄 Convirtiendo directorio → archivo .pth.tar (formato TAR)...")
        
        try:
            # SOLUCIÓN: Re-empaquetar como TAR (formato legacy que PyTorch entiende)
            # PyTorch espera que los archivos estén en la raíz del TAR, no en subdirectorio
            
            if os.path.exists(dest_path):
                os.remove(dest_path)
            
            # Crear archivo TAR con el contenido del directorio en la raíz
            import tarfile
            with tarfile.open(dest_path, 'w') as tar:
                # Agregar cada archivo/carpeta del checkpoint a la raíz del TAR
                for item in os.listdir(ckpt_path):
                    item_path = os.path.join(ckpt_path, item)
                    tar.add(item_path, arcname=item)
            
            # Verificar tamaño del archivo creado
            size_mb = os.path.getsize(dest_path) / (1024 * 1024)
            print(f"   ✓ Archivo creado: {final_name} ({size_mb:.1f} MB)")
            
            # Verificar que PyTorch puede cargar el archivo
            # Nota: La verificación puede fallar pero el modelo cargará correctamente
            try:
                test_load = torch.load(dest_path, map_location='cpu', weights_only=False)
                keys = list(test_load.keys())
                print(f"   ✓ Verificación exitosa - Keys: {keys}")
            except KeyError as e:
                # KeyError es esperado en algunos casos - el modelo lo manejará correctamente
                if 'storages' in str(e) or 'filename' in str(e):
                    print(f"   ⚠️  Advertencia (esperada): {type(e).__name__}: {str(e)[:100]}")
                    print(f"   💡 Esto es normal - el modelo usará su propio cargador legacy")
                else:
                    raise
            except Exception as e:
                print(f"   ⚠️  Advertencia al verificar: {type(e).__name__}: {str(e)[:100]}")
            
        except Exception as e:
            print(f"   ❌ ERROR al convertir checkpoint: {type(e).__name__}")
            print(f"      {str(e)[:200]}")
            import traceback
            print(f"   🔍 Traceback: {traceback.format_exc()[:300]}")
            shutil.rmtree(temp_dir, ignore_errors=True)
            return None
    
    # Limpiar temporal
    shutil.rmtree(temp_dir, ignore_errors=True)
    print(f"   ✓ Conversión completada")
    
    return epoch

# Extraer modelo L (epoch 83 según el paper)
print("="*60)
model_l_path = f'{MODELS_DATASET_PATH}/ConvNeXtPose_L.tar'
epoch_l = extract_checkpoint(model_l_path, 'L', expected_epoch=83)

print()

# Extraer modelo M (epoch 70 según el paper)
model_m_path = f'{MODELS_DATASET_PATH}/ConvNeXtPose_M.tar'
epoch_m = extract_checkpoint(model_m_path, 'M', expected_epoch=70)
print("="*60)

# Verificar checkpoints extraídos
print("\n📂 Checkpoints disponibles:")
checkpoints = [f for f in os.listdir('output/model_dump') 
               if f.endswith('.pth.tar') and os.path.isfile(os.path.join('output/model_dump', f))]

if checkpoints:
    for ckpt in sorted(checkpoints):
        ckpt_path = os.path.join('output/model_dump', ckpt)
        size_mb = os.path.getsize(ckpt_path) / (1024 * 1024)
        print(f"  ✓ {ckpt} ({size_mb:.1f} MB)")
        
        # Verificar contenido
        try:
            test_load = torch.load(ckpt_path, map_location='cpu')
            keys = list(test_load.keys())
            print(f"    → Keys: {keys}")
            if 'network' in test_load:
                print(f"    → ✅ Formato válido para torch.load()")
            else:
                print(f"    → ⚠️  Falta key 'network'")
        except Exception as e:
            print(f"    → ❌ Error al verificar: {type(e).__name__}: {str(e)[:80]}")

# Mostrar información de epochs
if epoch_l:
    print(f"\n💡 Modelo L: Usa CHECKPOINT_EPOCH = {epoch_l}")
if epoch_m:
    print(f"💡 Modelo M: Usa CHECKPOINT_EPOCH = {epoch_m}")

if epoch_l or epoch_m:
    print("\n✅ Los checkpoints están listos para torch.load()")
    print("   Formato: .pth.tar (TAR con estructura legacy)")
else:
    print("\n❌ No se pudieron extraer los checkpoints")
    print("💡 Verifica que los archivos .tar se descargaron correctamente")


### ⚠️ Nota sobre Extracción de Checkpoints

Los archivos `.tar` de ConvNeXtPose son en realidad **archivos ZIP** con estructura anidada:
```
ConvNeXtPose_L.tar (archivo zip)
└── snapshot_83.pth/        ← Directorio (formato legacy de PyTorch)
    ├── data.pkl            ← Metadatos del modelo (epoch, network, etc.)
    ├── version             ← Versión de PyTorch
    └── data/               ← Tensors del modelo
        ├── 0, 1, 2...      ← Fragmentos binarios
```

**Problema:** `torch.load()` en PyTorch moderno **NO acepta directorios**, solo archivos.

**Solución:** La celda anterior:
1. Extrae el contenido del ZIP
2. Lee `data.pkl` del directorio
3. Convierte y guarda como archivo `.pth` estándar que `torch.load()` puede abrir


In [ ]:
# Verificar checkpoints extraídos y detectar epoch
import glob
import re
import os
import torch

print("🔍 Verificación de Checkpoints:\n")

# Buscar ARCHIVOS .pth.tar
checkpoint_dir = 'output/model_dump'
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) 
                   if f.endswith('.pth.tar') and os.path.isfile(os.path.join(checkpoint_dir, f))]
else:
    checkpoints = []

if checkpoints:
    print("✅ Checkpoints encontrados:\n")
    for ckpt in sorted(checkpoints):
        ckpt_path = os.path.join(checkpoint_dir, ckpt)
        size_mb = os.path.getsize(ckpt_path) / (1024 * 1024)
        
        # Extraer epoch
        match = re.search(r'snapshot_(\d+)', ckpt)
        if match:
            epoch = match.group(1)
            print(f"  ✓ {ckpt} ({size_mb:.1f} MB)")
            print(f"    → Usa CHECKPOINT_EPOCH = {epoch}")
            
            # Verificar contenido del checkpoint
            try:
                test_load = torch.load(ckpt_path, map_location='cpu', weights_only=False)
                keys = list(test_load.keys())
                print(f"    → Keys: {keys}")
                
                if 'network' in test_load:
                    # Contar parámetros del modelo
                    num_params = sum(p.numel() for p in test_load['network'].values() 
                                     if isinstance(p, torch.Tensor))
                    print(f"    → ✅ Formato válido ({num_params:,} parámetros)")
                else:
                    print(f"    → ⚠️  Falta key 'network'")
                    
            except KeyError as e:
                # KeyError con 'storages' es esperado - el modelo lo manejará
                if 'storages' in str(e) or 'filename' in str(e):
                    print(f"    → ⚠️  KeyError (esperado para formato legacy)")
                    print(f"    → 💡 El modelo cargará esto correctamente en base.py")
                else:
                    print(f"    → ❌ Error inesperado: {type(e).__name__}: {str(e)[:50]}")
            except Exception as e:
                print(f"    → ❌ Error al verificar: {type(e).__name__}: {str(e)[:50]}")
            print()
else:
    print("❌ No se encontraron checkpoints .pth.tar válidos\n")
    
    # Diagnosticar problema
    if os.path.exists(checkpoint_dir):
        all_items = os.listdir(checkpoint_dir)
        if all_items:
            print("📂 Contenido de output/model_dump/:")
            for item in all_items[:15]:
                item_path = os.path.join(checkpoint_dir, item)
                if os.path.isdir(item_path):
                    print(f"    📁 {item}/ (directorio - no válido)")
                else:
                    size_mb = os.path.getsize(item_path) / (1024 * 1024)
                    print(f"    📄 {item} ({size_mb:.1f} MB)")
            
            print("\n💡 Solución: Re-ejecuta la celda anterior de extracción")
        else:
            print("💡 Directorio vacío - verifica MODELS_DATASET_PATH")
    else:
        print("💡 Solución: Verifica que MODELS_DATASET_PATH es correcto")

print("\n" + "="*60)
print("💡 IMPORTANTE: Los checkpoints deben ser archivos .pth.tar")
print("   (TAR con formato legacy que PyTorch puede leer)")
print("="*60)


**⚠️ IMPORTANTE:** Ajusta el valor de `CHECKPOINT_EPOCH` en las siguientes celdas según el epoch de tu checkpoint extraído. En este caso detectamos `snapshot_83.pth`, así que usa `CHECKPOINT_EPOCH = 83`.

## 🚀 PASO 4: Ejecutar Testing

### Modelo L (Large)

In [ ]:
# Diagnóstico: Verificar que todos los componentes están listos
import sys
import os

print("🔍 Verificación de Componentes:\n")

# 1. Dataset - Verificar estructura DENTRO del proyecto
project_root = '/kaggle/working/ConvNeXtPose'
data_dir = os.path.join(project_root, 'data')
h36m_path = os.path.join(data_dir, 'Human36M')

print(f"1. Dataset (estructura del proyecto):")
print(f"   Project root: {project_root}")
print(f"   data/ exists: {os.path.exists(data_dir)}")
print(f"   data/dataset.py: {os.path.exists(os.path.join(data_dir, 'dataset.py'))}")
print(f"   data/Human36M/ exists: {os.path.exists(h36m_path)}")

if os.path.exists(h36m_path):
    print(f"   - Human36M.py: {os.path.exists(os.path.join(h36m_path, 'Human36M.py'))}")
    print(f"   - annotations: {os.path.exists(os.path.join(h36m_path, 'annotations'))}")
    print(f"   - images/S9: {os.path.exists(os.path.join(h36m_path, 'images', 'S9'))}")
    print(f"   - images/S11: {os.path.exists(os.path.join(h36m_path, 'images', 'S11'))}")

# 2. Checkpoints
checkpoint_dir = os.path.join(project_root, 'output/model_dump')
print(f"\n2. Checkpoints: {checkpoint_dir}")
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('snapshot_')]
    if checkpoints:
        print(f"   Disponibles: {', '.join(checkpoints)}")
        # Extraer epoch del checkpoint
        import re
        for ckpt in checkpoints:
            match = re.search(r'snapshot_(\d+)', ckpt)
            if match:
                epoch = match.group(1)
                print(f"   💡 Usa CHECKPOINT_EPOCH = {epoch} en la siguiente celda")
    else:
        print("   ⚠️  No se encontraron checkpoints")
else:
    print("   ❌ Directorio no existe")

# 3. Estructura del proyecto
print(f"\n3. Estructura del proyecto:")
critical_files = ['main/config.py', 'common/base.py', 'data/dataset.py', 'data/Human36M/Human36M.py']
for file_path in critical_files:
    full_path = os.path.join(project_root, file_path)
    exists = os.path.exists(full_path)
    status = "✓" if exists else "❌"
    print(f"   {status} {file_path}")

print("\n" + "="*60)
if all(os.path.exists(os.path.join(project_root, f)) for f in critical_files):
    print("✅ Todos los checks pasaron - Listo para testing")
else:
    print("❌ Algunos archivos faltan - Revisa la configuración")
print("="*60)

### Ejecutar Testing desde Python

**Estructura correcta:**
- ✅ Módulos Python originales en `data/` (dataset.py, Human36M.py)
- ✅ Dataset de Kaggle enlazado en `data/Human36M/` (images, annotations)
- ✅ `config.py` configura automáticamente los paths

In [ ]:
# Testing con estructura correcta del proyecto
import sys
import os

# 1. Cambiar al directorio main (donde está config.py)
os.chdir('/kaggle/working/ConvNeXtPose/main')

# 2. Importar config PRIMERO - esto configura automáticamente los paths
from config import cfg

# 3. Cargar variante ANTES de importar otros módulos
VARIANT = 'L'  # Cambiar a 'M' para modelo Medium
CHECKPOINT_EPOCH = 83  # ← AJUSTAR según tu checkpoint

print(f"\n{'='*60}")
print(f"  Testing ConvNeXtPose-{VARIANT}")
print(f"{'='*60}\n")

cfg.load_variant_config(VARIANT)
cfg.set_args('0')  # GPU 0

# 4. AHORA importar los demás módulos (config ya configuró sys.path)
import torch
import torch.backends.cudnn as cudnn
from base import Tester
import numpy as np
from tqdm import tqdm

cudnn.benchmark = True
cudnn.deterministic = False
cudnn.enabled = True

# 5. Crear tester y ejecutar
tester = Tester()
tester._make_batch_generator()
tester._make_model(CHECKPOINT_EPOCH)

print(f"\n🚀 Ejecutando testing en epoch {CHECKPOINT_EPOCH}...\n")

preds = []
with torch.no_grad():
    for itr, input_img in enumerate(tqdm(tester.batch_generator)):
        coord_out = tester.model(input_img)
        
        if cfg.flip_test:
            from utils.pose_utils import flip
            flipped_input_img = flip(input_img, dims=3)
            flipped_coord_out = tester.model(flipped_input_img)
            flipped_coord_out[:, :, 0] = cfg.output_shape[1] - flipped_coord_out[:, :, 0] - 1
            for pair in tester.flip_pairs:
                flipped_coord_out[:, pair[0], :], flipped_coord_out[:, pair[1], :] = \
                    flipped_coord_out[:, pair[1], :].clone(), flipped_coord_out[:, pair[0], :].clone()
            coord_out = (coord_out + flipped_coord_out)/2.
        
        coord_out = coord_out.cpu().numpy()
        preds.append(coord_out)

# Evaluar
preds = np.concatenate(preds, axis=0)
print(f"\n📊 Evaluando {len(preds)} predicciones...\n")
tester._evaluate(preds, cfg.result_dir)

print(f"\n✅ Testing completado!")
print(f"📂 Resultados guardados en: {cfg.result_dir}")

### Modelo M (Medium) - Opcional

In [ ]:
# Si tienes el checkpoint del modelo M, ejecuta esto:
# !python test.py --gpu 0 --epochs {CHECKPOINT_EPOCH} --variant M

## 📊 PASO 5: Verificar Resultados

In [ ]:
# Ver resultados generados
%cd ..
!ls -lh output/result/

# Leer log de resultados
import glob
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    print(f"\n📄 Últimas líneas del log ({os.path.basename(latest_log)}):")
    !tail -n 20 {latest_log}
else:
    print("⚠️  No se encontraron logs")

## 📈 PASO 6: Análisis de Resultados

In [ ]:
import re

def extract_mpjpe_from_log(log_path):
    """Extrae el MPJPE del log"""
    with open(log_path, 'r') as f:
        content = f.read()
    
    # Buscar patrón de MPJPE
    pattern = r'MPJPE.*?(\d+\.\d+)'
    matches = re.findall(pattern, content)
    
    if matches:
        return float(matches[-1])  # Último valor
    return None

# Extraer resultados
log_files = glob.glob('output/log/*.log')
if log_files:
    latest_log = max(log_files, key=os.path.getctime)
    mpjpe = extract_mpjpe_from_log(latest_log)
    
    print("\n" + "="*60)
    print("  📊 RESULTADOS FINALES")
    print("="*60)
    
    if mpjpe:
        print(f"\n  MPJPE (Protocol 2): {mpjpe:.2f} mm")
        
        # Comparar con paper
        expected = {
            'L': 42.3,
            'M': 44.6
        }
        
        # Determinar variante
        for variant, expected_val in expected.items():
            diff = abs(mpjpe - expected_val)
            if diff < 5:
                print(f"\n  Variante detectada: {variant}")
                print(f"  Valor del paper: {expected_val:.2f} mm")
                print(f"  Diferencia: {mpjpe - expected_val:+.2f} mm")
                
                if diff < 2:
                    print("  ✅ Resultado excelente (dentro de ±2mm)")
                elif diff < 5:
                    print("  ✓ Resultado aceptable (dentro de ±5mm)")
                break
    else:
        print("\n⚠️  No se pudo extraer MPJPE del log")
        print("Revisa el log manualmente en output/log/")
    
    print("\n" + "="*60)
else:
    print("❌ No se encontraron logs de testing")

## 💾 PASO 7: Guardar Outputs

Kaggle guarda automáticamente todo en `/kaggle/working/`. Opcionalmente puedes copiar resultados específicos:

In [ ]:
# Crear resumen de resultados
import json
from datetime import datetime

summary = {
    'timestamp': datetime.now().isoformat(),
    'model': 'ConvNeXtPose-L',  # Cambiar según modelo testeado
    'checkpoint_epoch': CHECKPOINT_EPOCH,
    'dataset': 'Human3.6M Protocol 2',
    'mpjpe_mm': mpjpe if 'mpjpe' in locals() else None,
    'pytorch_version': torch.__version__,
    'cuda_version': torch.version.cuda if torch.cuda.is_available() else None
}

# Guardar resumen
with open('output/result/test_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Resumen guardado en output/result/test_summary.json")
print("\n📄 Contenido:")
print(json.dumps(summary, indent=2))

## 🎉 Testing Completado!

Los resultados están en:
- **Logs**: `output/log/`
- **Resultados**: `output/result/`
- **Resumen JSON**: `output/result/test_summary.json`

Todos los archivos en `/kaggle/working/ConvNeXtPose/output/` se guardarán automáticamente cuando hagas commit del notebook.